In [1]:
import numpy as np
import scipy.optimize as optimize

In [2]:
""" Optimization Algorithm """
""" New Matrix """
def newMat(x, Vt, k):
  V_new = np.zeros((Vt.shape), dtype=np.cfloat)
  if k==2:
    V_new[0] = np.cos(x[0])
    V_new[1] = (np.sin(x[0])) * np.exp(1j*x[1])
  elif k==3:
    V_new[0] = np.cos(x[0])
    V_new[1] = (np.sin(x[0])) * (np.cos(x[1])) * np.exp(1j*x[2])
    V_new[2] = (np.sin(x[0])) * (np.sin(x[1])) * np.exp(1j*x[3])
  elif k==4:
    V_new[0] = (np.cos(x[0])) * (np.cos(x[1]))
    V_new[1] = (np.cos(x[0])) * (np.sin(x[1])) * np.exp(1j*x[3])
    V_new[2] = (np.sin(x[0])) * (np.cos(x[2])) * np.exp(1j*x[4])
    V_new[3] = (np.sin(x[0])) * (np.sin(x[2])) * np.exp(1j*x[5])
  elif k==5:
    V_new[0] = (np.cos(x[0])) * (np.cos(x[1]))
    V_new[1] = (np.cos(x[0])) * (np.sin(x[1])) * np.exp(1j*x[3])
    V_new[2] = (np.sin(x[0])) * (np.cos(x[2])) * np.exp(1j*x[4])
    V_new[3] = (np.sin(x[0])) * (np.sin(x[2])) * (np.sin(x[6])) * np.exp(1j*x[5])
    V_new[4] = (np.sin(x[0])) * (np.sin(x[2])) * (np.cos(x[6])) * np.exp(1j*x[7])
  else:
    V_new[0] = (np.cos(x[0])) * (np.cos(x[1]))
    V_new[1] = (np.cos(x[0])) * (np.sin(x[1])) * np.exp(1j*x[3])
    V_new[2] = (np.sin(x[0])) * (np.cos(x[2])) * np.exp(1j*x[4])
    V_new[3] = (np.sin(x[0])) * (np.sin(x[2])) * (np.sin(x[6])) * np.exp(1j*x[5])
    V_new[4] = (np.sin(x[0])) * (np.sin(x[2])) * (np.cos(x[6])) * np.exp(1j*x[7])
    V_new[5] = (np.sin(x[0])) * (np.sin(x[2])) * (np.sin(x[6])) * (np.cos(x[8])) * np.exp(1j*x[9])
  return V_new


""" Cost Function """
def costFn(x, Ut, Vt, A, k):
    V_new = newMat(x, Vt, k)
    Bp = np.dot(Ut, V_new) 
    loss = np.linalg.norm(A - Bp*np.conjugate(Bp))
    return (loss)


In [3]:
def calcResults(k, m, n):
    print ("m = ",m,", n = ",n)
    res = np.zeros((100,2))
    for i in range(100):
        A = np.random.rand(m, n)
        A = A/A.sum(axis=0)         # Optimize column-wise

        # Classic Truncated SVD
        U, L, V = np.linalg.svd(A, full_matrices=False)
        Ut = U[:, :k]
        Vt = V[:k]
        Lt = L[:k]
        At = np.dot(np.dot(Ut,np.diag(Lt)), Vt)
        res[i][0] = (np.linalg.norm(A - At))


        # Complex SVD
        B = np.sqrt(A)
        U, L, V = np.linalg.svd(B, full_matrices=False)
        Ut = U[:, :k]
        Vt = V[:k]
        Lt = L[:k]
        initial_guess = np.ones((2*(k-1),), dtype=np.longdouble)
        V_new = np.zeros(Vt.shape, dtype=np.cfloat)
        for col in range(Vt.shape[1]):
            result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut,Vt[:, col],A[:,col],k),
                                    tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
            V_new[:,col] = newMat(result.x, Vt[:, col], k)
        Bp = np.dot(Ut, V_new)  
        res[i][1] = (np.linalg.norm(A - np.conjugate(Bp)*Bp))


        if i%10==0: print(i, end=' ')
    print('\n')
    return res

In [4]:
res = calcResults(k=6, m=7, n=7)
print(res)

m =  7 , n =  7
0 10 20 30 40 50 60 70 80 90 

[[6.55132440e-04 9.09690467e-03]
 [1.42470236e-02 7.33092021e-04]
 [2.09237903e-02 7.24112984e-08]
 [6.03475882e-02 3.87681241e-13]
 [5.98942966e-03 1.24810365e-03]
 [3.07914787e-02 4.54050227e-04]
 [3.75357985e-02 1.32953926e-02]
 [2.87612196e-02 6.02634504e-03]
 [5.96832053e-02 2.52788523e-02]
 [3.72442949e-02 1.65572156e-03]
 [1.48736411e-02 2.41901349e-03]
 [2.21902540e-02 2.96909283e-03]
 [2.41905515e-02 4.66384139e-03]
 [2.73693218e-02 7.03540045e-03]
 [1.59952101e-02 7.93855891e-04]
 [1.22814772e-03 2.00040753e-04]
 [3.00017286e-02 1.39263118e-04]
 [7.84346223e-03 2.92627848e-03]
 [4.00788133e-02 8.27379724e-13]
 [1.62943346e-02 1.64785573e-03]
 [1.88370271e-02 2.84936862e-04]
 [2.47436249e-03 2.66911752e-04]
 [8.06887847e-03 1.43158409e-03]
 [1.75261262e-02 1.71717377e-04]
 [2.13185838e-02 1.58827977e-03]
 [5.62938552e-02 6.60497425e-03]
 [1.73659152e-02 3.04141736e-03]
 [1.98467924e-02 1.71859784e-03]
 [3.64554151e-02 1.40191586e-

In [5]:
res.mean(axis=0)

array([0.02306244, 0.00371709])